# Term Proximity Retrieval Model in SQL using DuckDB and MonetDBLite

This notebook is a work in progress for the Information Retrieval course research project. A number of retrievel models are implemented in SQL queries that can be run with two database systems.

## Index

A preliminary index of the Robust04 data set was first [downloaded from Jimmy Lin's Dropbox](https://www.dropbox.com/s/mdoly9sjdalh44x/lucene-index.robust04.pos%2Bdocvectors%2Brawdocs.tar.gz). Then the [OldDog](https://github.com/Chriskamphuis/olddog) code from Chris Kamphuis and Arjen de Vries was modified to work with more than one leaf. Their code was further modified to store the collection frequency of each term and the term frequency of each term in each document. The modified code was run, which resulted in three CSV tables. 

The *dict* table houses termid, term, document frequency, and collection frequency data. The *docs* table houses name, docid, and document length information. And finally, the *terms* table houses the termid, docid, position, and term frequency data.

These three tables are put in an archive, so that they can be easily [downloaded from Dropbox](https://www.dropbox.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar).

In [0]:
!wget -O Robust04.rar https://dl.dropboxusercontent.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar?dl=0
!unrar e -o+ Robust04.rar

In [0]:
from datetime import datetime

## Databases

Both DuckDB and MonetDBLite will be used to test the differences between them on a number of dimensions, during the evaluation phase of the research.

Google Colab does not pre-install these packages, so that is why we need to do `pip install`. This takes about 5 minutes for DuckDB, so please already run the next cell, before reading on.

Here, we have made two classes that do the necessary tasks that we need the databases to do.

MonetDB cannot be used, as it requires an external Java MonetDB server, and the project was to be made with the Python APIs. That leaves us with MonetDBLite. For some reason, they have taken out the Python DB API for MonetDBLite (the `Cursor` class and `cursor.execute()` function). We could enable it by modifying the source code, but the program also needs to work for other people on other computers. So we just stuck to the Simple API of MonetDBLite. Another titbit, MonetDBLite cannot be initialized in the memory. If you go over some memory bandwith threshold, the whole database stops working. So, MonetDBLite needs to be initialized in storage. The performance difference is not that big on Google Colab, but it did matter on a local runtime.

When initializing the database objects, the index tables are automatically added. So you only have to initialize them once.

In [0]:
!pip install duckdb
import duckdb

class DuckDB(object):
  """
  Class that houses all the DuckDB functionalities.

  Attributes:
    c         = [Cursor] database cursor of DuckDB
    C         = [int] number of indexed terms
    N         = [int] number of indexed documents
    avgdl     = [float] average number of terms per document
    len_query = [int] number of terms in current search query
  """
  def __init__(self,
               database=':memory:',
               dict='dict.csv',
               docs='docs.csv',
               terms='terms.csv'):
    """
    Initializes DuckDB database with index and statistics.

    Args:
      database = [str] database path
      dict     = [str] filename for dictionary CSV
      docs     = [str] filename for documents CSV
      terms    = [str] filename for terms CSV
    """
    # initialize database
    con = duckdb.connect(database)
    self.c = con.cursor()

    # copy dictionary CSV into DuckDB database
    self.c.execute("CREATE TABLE dict(termid INTEGER "
                                    ",term   VARCHAR "
                                    ",df     INTEGER "
                                    ",cf     INTEGER)")
    self.c.execute("COPY dict "
                   "FROM '" + dict + "' "
                   "WITH DELIMITER '|'")
    
    # copy documents CSV into DuckDB database
    self.c.execute("CREATE TABLE docs(name  VARCHAR "
                                    ",docid INTEGER "
                                    ",len   INTEGER "
                                    ",temp  INTEGER)")
    self.c.execute("COPY docs "
                   "FROM '" + docs + "' "
                   "WITH DELIMITER '|'")
    
    # copy terms CSV into DuckDB database
    self.c.execute("CREATE TABLE terms(termid INTEGER "
                                     ",docid  INTEGER "
                                     ",pos    INTEGER "
                                     ",tf     INTEGER)")
    self.c.execute("COPY terms "
                   "FROM '" + terms + "' "
                   "WITH DELIMITER '|'")
    
    # compute standard index statistics
    self.C = self._C()
    self.N = self._N()
    self.avgdl = self._avgdl()
    self.len_query = 0
    
  def make_query(self, *args: str):
    """
    Makes query table in DuckDB database filled with query terms.
    
    Args:
      args = [[str]] concatenation of strings to be made into a query
    """
    # convert search query in to SQL query
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    # make new or replace old query table
    self.c.execute("DROP TABLE IF EXISTS query")
    self.c.execute("CREATE TABLE query(term VARCHAR)")
    self.c.execute("INSERT INTO query VALUES " + query)

    # bookkeeping
    self.len_query = len(args)

  def execute_query(self, query):
    """
    Executes SQL query on DuckDB database.

    Args:
      query = [str] the SQL query to be executed by DuckDB

    Returns [DataFrame]:
      The output of the execution as a Pandas DataFrame object.
    """
    out = self.c.execute(query)
    return out.fetchdf()

  def _C(self):
    """ 
    Gets total number of terms in the index.
    
    Returns [int]:
      Total number of indexed terms.
    """
    C = self.c.execute("SELECT SUM(dict.cf) "
                       "FROM dict")
    return C.fetchdf().iloc[0, 0]

  def _N(self):
    """
    Gets number of documents in the index.

    Returns [int]:
      Number of indexed documents.
    """
    N = self.c.execute("SELECT COUNT(*) "
                       "FROM docs")
    return N.fetchdf().iloc[0, 0]

  def _avgdl(self):
    """
    Gets average number of terms per document in the index.

    Returns [float]:
      Average length of indexed documents.
    """
    avgdl = self.c.execute("SELECT AVG(docs.len) "
                           "FROM docs")
    return avgdl.fetchdf().iloc[0, 0]

In [0]:
!pip install monetdblite
import monetdblite as m
import pandas as pd

class MonetDBLite(object):
  """ 
  Class that houses all the MonetDBLite functionalities. 

  Attributes:
    C     = [int] number of indexed terms
    N     = [int] number of indexed documents
    avgdl = [float] average number of terms per document
    len_query = [int] number of terms in current search query
  """
  def __init__(self,
               database='/tmp/MonetDBLite Database',
               dict='/content/dict.csv',
               docs='/content/docs.csv',
               terms='/content/terms.csv'):
    """
    Initializes MonetDBLite database with index.

    Args:
      database = [str] database path
      dict     = [str] filename for dictionary CSV
      docs     = [str] filename for documents CSV
      terms    = [str] filename for terms CSV
    """
    # initialize database
    m.init(database)

    # copy dictionary CSV into MonetDBLite database
    m.sql("CREATE TABLE dict(termid INTEGER "
                           ",term   VARCHAR(99) "
                           ",df     INTEGER "
                           ",cf     INTEGER)")
    m.sql("COPY INTO dict "
          "FROM '" + dict + "' "
          "USING DELIMITERS '|'")
    
    # copy documents CSV into MonetDBLite database
    m.sql("CREATE TABLE docs(name  VARCHAR(99) "
                           ",docid INTEGER "
                           ",len   INTEGER "
                           ",temp  INTEGER)") 
    m.sql("COPY INTO docs "
          "FROM '" + docs + "' "
          "USING DELIMITERS '|'")
    
    # copy terms CSV into MonetDBLite database
    m.sql("CREATE TABLE terms(termid INTEGER "
                            ",docid  INTEGER "
                            ",pos    INTEGER "
                            ",tf     INTEGER)")
    m.sql("COPY INTO terms "
          "FROM '" + terms + "' "
          "USING DELIMITERS '|'")
    
    # compute standard index statistics
    self.C = self._C()
    self.N = self._N()
    self.avgdl = self._avgdl()
    self.len_query = 0
    
  def make_query(self, *args: str):
    """
    Makes query table in MonetDBLite database filled with query terms.
    
    Args:
      args = [[str]] concatenation of strings to be made into a query
    """
    # convert search query in to SQL query
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    # make new or replace old query table
    m.sql("DROP TABLE IF EXISTS query")
    m.sql("CREATE TABLE query(term VARCHAR(99))")
    m.sql("INSERT INTO query VALUES " + query)

    # bookkeeping
    self.len_query = len(args)

  def execute_query(self, query):
    """
    Executes SQL query on MonetDBLite database.

    Args:
      query = [str] the SQL query to be executed by MonetDBLite

    Returns [DataFrame]:
      The output of the execution as a Pandas DataFrame object.
    """
    out = m.sql(query)
    return pd.DataFrame.from_dict(out)

  def _C(self):
    """ 
    Gets total number of terms in the index.
    
    Returns [int]:
      Total number of indexed terms.
    """
    C = m.sql("SELECT SUM(dict.cf) "
              "FROM dict")
    return pd.DataFrame.from_dict(C).iloc[0, 0]

  def _N(self):
    """
    Gets number of documents in the index.

    Returns [int]:
      Number of indexed documents.
    """
    N = m.sql("SELECT COUNT(*) "
              "FROM docs")
    return pd.DataFrame.from_dict(N).iloc[0, 0]

  def _avgdl(self):
    """
    Gets average number of terms per document in the index.

    Returns [float]:
      Average length of indexed documents.
    """
    avgdl = m.sql("SELECT AVG(docs.len) "
                  "FROM docs")
    return pd.DataFrame.from_dict(avgdl).iloc[0, 0]

## Retriever

The next class houses the function for retrieving the relevant documents given a number of options, `retrieve()`. The private methods (starting with an underscore) could really use some help. So please only use the `retrieve()` function further into the file. Perhaps we will clean up the mess later.

In [0]:
class Retriever(object):
  """ Class to do document retrieval with term proximity using databases. """     
  def retrieve(self,
               query, 
               db,
               con_query=True, 
               pre_select='kld', 
               tp=True,
               k=30,
               sum=True,
               mu=0.8, # totally not sure about this hyper-parameter
               k1=1.2,
               b=0.75,
               num_docs=20,
               max_span=5):
    """
    Function that retreives documents with a Retrieval Status Value (RSV)
    based on term-proximity (TP) weighting, Okapi BM25 or Kullback-Leibler
    Divergence. When opting for TP, k documents can be pre-selected with
    the Okapi BM25 or Kullback-Leibler Divergence retrieval models.

    Args:
      query      = [[str]] the tokenized and normalied query
      db         = [DuckDB|MonetDBLite] database that stores the index
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      tp         = [bool] whether to do the term proximity at all
      k          = [int] maximum number of documents to retrieve with
                         the pre-selection retrieval model
      sum        = [bool] whether to sum the pre-selection and term 
                          proximity scores for the final score
      mu         = [float] hyper-parameter for the KLD retrieval model
      k1         = [float] hyper-parameter for Okapi BM25
      b          = [float] hyper-parameter for Okapi BM25
      num_docs   = [int] maximum number of documents to retrieve
      max_span   = [int] maximum distance, in number of terms, for a term
                         pair to be included in the term proximity score

    Returns [DataFrame]:
      The Pandas DataFrame output.
    """
    # add the search query as a table to the database
    db.make_query(*query)

    # determine the SQL query
    sql = (self._qterms(pre_select, tp) +
           self._qtermstf(pre_select, tp) +
           self._condocs(db, con_query, tp) +
           self._pre_select_subscores(db, con_query, pre_select, mu, k1, b) +
           self._topkdocs(pre_select, k) + 
           self._pairs(con_query, pre_select, tp, max_span) +
           self._tpscores(db, tp, k1, b) +
           self._scores(pre_select, tp, sum, num_docs))
    print('Query: {}'.format(sql))
    
    # get the elapsed time and the results after executing the SQL query
    time = datetime.now()
    out = db.execute_query(sql)
    time_delta = datetime.now() - time
    print('Query time: {}'.format(time_delta))

    return out

  def _qterms(self, pre_select, tp):
    """ 
    Get the SQL query that will retrieve the rows in the terms file
    belonging to the query terms, including the positional information.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model;
                   Kullback-Leibler Divergence retrieval model also needs
                   collection frequency information of each term  
      tp         = [bool] whether to do the term proximity

    Returns [str]:
      SQL query as string.
    """
    query = ("WITH qtermids "
                  "AS (SELECT dict.termid "
                            ",dict.df "
                            "{}"
                      "FROM dict "
                      "JOIN query "
                      "ON dict.term = query.term"
                      ") "
             "{}")
    
    if tp:
      query = query.format("{}",
                           ", qterms "
                           "AS (SELECT terms.termid "
                                     ",terms.docid "
                                     ",terms.pos "
                                     ",terms.tf "
                                     ",qtermids.df "
                                     "{}"
                               "FROM terms "
                               "JOIN qtermids "
                               "ON terms.termid = qtermids.termid"
                               ") ")
      if pre_select == 'kld':
        return query.format(",dict.cf ", ",qtermids.cf ")
      else:
        return query.format("", "")
    else:
      if pre_select == 'kld':
        return query.format(",dict.cf ", "")
      else:
        return query.format("", "")  

  def _qtermstf(self, pre_select, tp):
    """
    Get the SQL query that will retrieve the rows in the terms file
    belonging to the query terms, excluding the positional information.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model;
                   Kullback-Leibler Divergence retrieval model also needs
                   collection frequency information of each term
      tp         = [bool] whether to do the term proximity

    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'none':
      return ""
    
    query = (", qtermstfrows "
                  "AS (SELECT qterms.termid "
                            ",qterms.docid "
                            ",qterms.tf "
                            "{}"
                            "{}"
                            ",( ROW_NUMBER() "
                               "OVER(PARTITION BY qterms.termid, qterms.docid "
                                    "ORDER BY qterms.pos"
                                    ")"
                              ") AS row "
                      "{}" 
                      ") "
             ", qtermstf "
                  "AS (SELECT qtermstfrows.termid "
                            ",qtermstfrows.docid "
                            ",qtermstfrows.tf "
                            ",qtermstfrows.df "
                            "{}"
                      "FROM qtermstfrows "
                      "WHERE qtermstfrows.row = 1"
                      ") ")

    if pre_select == 'kld':
      if tp:
        return query.format(",qterms.df ",
                            ",qterms.cf ",
                            "FROM qterms",
                            ",qtermstfrows.cf ")
      else:
        return query.format(",qtermids.df ",
                            ",qtermids.cf ",
                            "FROM terms AS qterms "
                            "JOIN qtermids "
                            "ON qterms.termid = qtermids.termid",
                            ",qtermstfrows.cf ")
    elif pre_select == 'okapi':
      if tp:
        return query.format(",qterms.df ",
                            "",
                            "FROM qterms",
                            "")
      else:
        return query.format(",qtermids.df ",
                            "",
                            "FROM terms AS qterms "
                            "JOIN qtermids "
                            "ON qterms.termid = qtermids.termid",
                            "")

  def _condocs(self, db, con_query, tp):
    """
    Get the SQL query that will retrieve the rows in the terms file
    belonging to documents that contain all the query terms.

    Args:
      db        = [DuckDB|MonetDBLite] database that stores the index
      con_query = [bool] whether all query terms need to be in
                         the document for it to be retrieved
      tp         = [bool] whether to do the term proximity

    Returns [str]:
      SQL query as string.                         
    """
    if con_query:
      query = (", condocs "
                    "AS (SELECT qterms.docid "
                        "FROM {} AS qterms "
                        "GROUP BY qterms.docid "
                        "HAVING COUNT(DISTINCT qterms.termid) = {:d}"
                        ") ")
      if tp:
        return query.format("qterms", db.len_query)
      else:
        return query.format("qtermstf", db.len_query)
    else:
      return ""

  def _pre_select_subscores(self, db, con_query, pre_select, mu, k1, b):
    """
    Get the SQL query that will compute a score for each 
    query term-document pair, according to the pre-selection
    retrieval model.

    Args:
      db         = [DuckDB|MonetDBLite] database that stores the index
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved     
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      mu         = [float] hyper-parameter for the KLD retrieval model
      k1         = [float] hyper-parameter for Okapi BM25
      b          = [float] hyper-parameter for Okapi BM25
    
    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'kld':
      query = (", kldsubscores "
                    "AS (SELECT qtermstf.docid "
                              ",( LOG({:f}+tf*{:f}/cf)"              
                                  "+" 
                                 "LOG(1/({:f}+len))"
                                ") AS subscore "
                        "FROM qtermstf "
                        "{}"
                        "JOIN docs "
                        "ON qtermstf.docid = docs.docid"
                        ") ")
      query = query.format(mu, db.C, mu, "{}")
    elif pre_select == 'okapi':
      query = (", okapisubscores "
                    "AS (SELECT qtermstf.docid "
                              ",( LOG(({:f}-df+0.5)/(df+0.5))*tf*({:f}+1)"
                                  "/"
                                 "(tf+{:f}*(1-{:f}+{:f}*len/{:f}))"
                                ") AS subscore "
                        "FROM qtermstf "
                        "{}"
                        "JOIN docs "
                        "ON qtermstf.docid = docs.docid"
                        ") ")
      query = query.format(db.N, k1, k1, b, b, db.avgdl, "{}")
    else:
      return ""

    if con_query:
      return query.format("JOIN condocs "
                          "ON qtermstf.docid = condocs.docid ")
    else:
      return query.format("")

  def _topkdocs(self, pre_select, k):
    """
    Get the SQL query that will compute the pre-selection scores,
    according to the pre-selection retrieval model, and retrieve
    the top k documents.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      k          = [int] maximum number of documents retrieved with
                       the pre-selection retrieval model
    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'none':
      return ""

    query = (", topdocs "
                  "AS (SELECT subscores.docid "
                            ",SUM(subscores.subscore) AS score "
                            ",( ROW_NUMBER() "
                                "OVER(ORDER BY SUM(subscores.subscore) DESC)"
                              ") AS row "
                      "FROM {} AS subscores "
                      "GROUP BY subscores.docid"
                      ") "
              ", topkdocs "
                  "AS (SELECT topdocs.docid "
                            ",topdocs.score "
                      "FROM topdocs "
                      "WHERE topdocs.row BETWEEN 1 AND {:d}"
                      ") ")
    
    if pre_select == 'kld':
      return query.format("kldsubscores", k)
    elif pre_select == 'okapi':
      return query.format("okapisubscores", k)

  def _pairs(self, con_query, pre_select, tp, max_span):
    """
    Get the SQL query that will compute the term pair instance (tpi) for
    each query term pair within a span of max_span terms.

    Args:
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved                          
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model  
      tp         = [bool] whether to do the term proximity                   
      max_span   = [int] the maximum span, in terms, of a term pair to
                        include in the term proximity score

    Returns [str]:
      SQL query as string.
    """
    if not tp:
      return ""

    query = (", pairs "
                  "AS (SELECT qterms1.termid AS termid1 "
                            ",qterms2.termid AS termid2 "
                            ",qterms1.docid "
                            ",1.0/(qterms1.pos-qterms2.pos) AS tpi "
                            ",( CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                    "ELSE qterms2.df "
                               "END"
                              ") AS maxdf "
                      "FROM qterms AS qterms1 "
                      "{}"
                      "{}"
                      "JOIN qterms AS qterms2 "
                      "ON qterms1.docid = qterms2.docid AND "
                         "NOT qterms1.termid = qterms2.termid AND "
                         "qterms1.pos - qterms2.pos BETWEEN 1 AND {:d}"
                      ") ")
    if con_query:
      query = query.format("JOIN condocs ON qterms1.docid = condocs.docid ",
                           "{}",
                           max_span)
    else:
      query = query.format("",
                           "{}",
                           max_span)
      
    if pre_select == 'none':
      return query.format("")
    else:
      return query.format("JOIN topkdocs ON qterms1.docid = topkdocs.docid ")
      
  def _tpscores(self, db, tp, k1, b):
    """
    Get the SQL query that will compute the term proximity score.

    Args:      
      db        = [DuckDB|MonetDBLite] database that stores the index
      tp       = [bool] whether to do the term proximity
      k1       = [float] hyper-parameter for Okapi BM25
      b        = [float] hyper-parameter for Okapi BM25

    Returns [str]:
      SQL query as string.
    """
    if not tp:
      return ""
      
    query = (",tpisums "
                  "AS (SELECT pairs.termid1 "
                            ",pairs.termid2 "
                            ",pairs.docid "
                            ",SUM(pairs.tpi) AS tpisum "
                      "FROM pairs "              
                      "GROUP BY pairs.termid1 "
                              ",pairs.termid2 "
                              ",pairs.docid"
                      ") "
            ", tpsubscores "
                  "AS (SELECT pairs.docid "
                            ",( LOG(({:f}-maxdf+0.5)/(maxdf+0.5))*tpisum*({:f}+1)"
                                "/"
                              "(tpisum+{:f}*(1-{:f}+{:f}*len/{:f}))"
                              ") AS tpsubscore "
                      "FROM pairs "
                      "JOIN tpisums "
                      "ON pairs.termid1 = tpisums.termid1 AND "
                        "pairs.termid2 = tpisums.termid2 AND "
                        "pairs.docid = tpisums.docid "
                      "JOIN docs "
                      "ON pairs.docid = docs.docid"
                      ") "
            ", tpscores "
                  "AS (SELECT tpsubscores.docid "
                            ",SUM(tpsubscores.tpsubscore) AS tpscore "
                      "FROM tpsubscores "
                      "GROUP BY tpsubscores.docid"
                      ") ")
    return query.format(db.N, k1, k1, b, b, db.avgdl)

  def _scores(self, pre_select, tp, sum, num_docs):
    """ 
    Get the SQL query that will retrieve or compute the final document scores.

    Args:      
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model 
      tp         = [bool] whether to do the term proximity
      sum        = [bool] whether to sum the pre-select and term 
                          proximity scores for the final score
      num_docs   = [int] maximum number of documents to retrieve

    Returns [str]:
      SQL query as string.
    """
    query = (", scores "
                  "AS ({}) "
             "SELECT scores.docid "
                   ",scores.score "
             "FROM scores "
             "ORDER BY scores.score DESC "
             "LIMIT {:d}")
    
    if not tp:
      return query.format("SELECT topkdocs.docid "
                                ",topkdocs.score "
                          "FROM topkdocs",
                          num_docs)
    elif pre_select == 'none' or not sum:
      return query.format("SELECT tpscores.docid "
                                ",tpscores.tpscore AS score "
                          "FROM tpscores",
                          num_docs)
    else:
      return query.format("SELECT topkdocs.docid "
                                ",( topkdocs.score"
                                    "+"
                                   "COALESCE(tpscores.tpscore, 0)"
                                  ") AS score "
                          "FROM topkdocs "
                          "LEFT JOIN tpscores "
                          "ON topkdocs.docid = tpscores.docid",
                          num_docs)

## Initialization

Both databases are initialized, which loads the indices in the databases and makes them ready for execution. This can take about 5 minutes, so time for coffee!

In [0]:
duck = DuckDB()
monet = MonetDBLite()

The `Retriever` class does not have a constructor, so initializing it is a bit meaningless. I still prefer to put it in a class to hide the private methods.

In [0]:
retriever = Retriever()

## Options

Now we will explain how to use this complicated `retrieve()` function by setting a number of specific options on or off.

### All options

Let's start with the default options. The query will be (`query=`)*new york* and it will be executed for both databases (`db=DuckDB` and `db=MonetDBLite`).

First, the top (`k=`)30 documents are retrieved using the Kullback-Leibler Divergence (`pre_select='kld'`) retrieval model with conjunctive queries (`con_query=True`) and a $\mu$ of 0.8 (`mu=0.8`). Then, these 30 documents are scored again with term proximity weighting (`tp=True`) based on a modified version of the [Rasolofo algorithm](https://www.researchgate.net/publication/225174089_Term_Proximity_Scoring_for_Keyword-Based_Retrieval_Systems), with a $k1$ of 1.2 (`k1=1.2`), a $b$ of 0.75 (`b=0.75`), and a maximum distance of 5 (`max_span=5`) between query terms. The scores obtained from KLD and Rasolofo are summed (`sum=True`) to arrive at the final score and ranking. Of this final ranking, 20 documents are retrieved (`num_docs=20`).

In [0]:
query = ['new', 'york']

print("DuckDB with all options")
duck_scores = retriever.retrieve(query, duck)

print("\nMonetDBLite with all options")
monet_scores = retriever.retrieve(query, monet)

DuckDB was very slow compared to MonetDBLite (approximately a factor of 35). The query that both databases execute is identical, so the time difference is purely DuckDB's shortcoming. Furthermore, the actual scores from DuckDB are different than from MonetDBLite. However, the ranking, surprisingly, is identical between the two.

In [0]:
duck_scores

In [0]:
monet_scores

### No term proximity weighting

If you only want to rank the documents based on Okapi BM25, then run the following code. All the other options, where applicable, are still the same.

In [0]:
print("DuckDB with only Okapi BM25")
duck_scores = retriever.retrieve(query, duck, pre_select='okapi', tp=False)

print("\nMonetDBLite with  only Okapi BM25")
monet_scores = retriever.retrieve(query, monet, pre_select='okapi', tp=False)

The difference in time between DuckDB and MonetDBLite is now much smaller (a factor of about 10), but the scores are still different, whereas the ranking is again identical.

In [0]:
duck_scores

In [0]:
monet_scores

### Only term proximity weighting

If you do not want to pre-select `k` documents, before computing the Rasolofo score, then run the following code.

In [0]:
print("DuckDB with only Rasolofo")
duck_scores = retriever.retrieve(query, duck, pre_select='none')

print("\nMonetDBLite with only Rasolofo")
monet_scores = retriever.retrieve(query, monet, pre_select='none')

The factor in the time difference is approximately 10. The execution time for MonetDBLite is more than previously. This makes sense; the algorithm needs to find the close query term pairs in all the documents instead of only the top 30 documents. This operation has a super-linear running time, so it takes MonetDBLite longer to produce the output.

The scores are again different, but now the ranking is also different. So DuckDB does not work for the given query.

In [0]:
duck_scores

In [0]:
monet_scores

### Bigger term pair radius

If the Rasolofo algorithm retrieves too few documents with the default parameters, the span of a term pair can be expanded to include more documents. To do that, run the below code with the query *wizard hat*.

It is slightly slower than the default span with MonetDBLite.

In [0]:
query = ['wizard', 'hat']

print("DuckDB with default term pair span")
duck_scores = retriever.retrieve(query, duck, pre_select='none')

print("\nMonetDBLite with default term pair span")
monet_scores = retriever.retrieve(query, monet, pre_select='none')

In [0]:
duck_scores

In [0]:
monet_scores

In [0]:
print("DuckDB with bigger term pair span")
duck_scores = retriever.retrieve(query, duck, pre_select='none', max_span=20)

print("\nMonetDBLite with bigger term pair span")
monet_scores = retriever.retrieve(query, monet, pre_select='none', max_span=20)

In [0]:
duck_scores

In [0]:
monet_scores

### Only TP scores

If you want that the final score is only the Rasolofo score, then set `sum=False`. So not the sum of the pre-selection score and the Rasolofo score as final score, but only the Rasolofo score. If the Rasolofo algorithm could not score a document that was retrieved by the pre-selection retrieval model, then that document will not be included in the ranking. So do be aware of that when setting this option!

In [0]:
print("DuckDB with summed scores")
duck_scores = retriever.retrieve(query, duck)

print("\nMonetDBLite with summed scores")
monet_scores = retriever.retrieve(query, monet)

In [0]:
duck_scores

In [0]:
monet_scores

In [0]:
print("DuckDB with summed scores")
duck_scores = retriever.retrieve(query, duck, sum=False)

print("\nMonetDBLite with summed scores")
monet_scores = retriever.retrieve(query, monet, sum=False)

In [0]:
duck_scores

In [0]:
monet_scores

## Conclusion

As seen by the many options, there are endless opportunities for research and evaluation. And we haven't even talked about combining options, disjunctive queries, or hyper-parameters, yet.

Our question right now is basically: why is DuckDB so slow and why does DuckDB give incorrect and unintuitive results. Analyzing the runtime of MonetDBLite was much more straightforward than for DuckDB. Is there still some hope for DuckDB, maybe in the future? Or should we only focus on MonetDBLite and maybe implement a second term proximity retrieval model, instead of comparing the two databases?